<a href="https://colab.research.google.com/github/Debottam/MachinLearningEx/blob/master/LSHash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

In [0]:
vec1 = np.random.rand(5)
vec2 = np.random.rand(5)
vec3 = np.random.rand(5)

In [9]:
vec1, vec2, vec3

(array([0.17910685, 0.09269596, 0.6923277 , 0.13269754, 0.61931283]),
 array([0.520308  , 0.52719017, 0.84547822, 0.34036769, 0.48015719]),
 array([0.09722213, 0.04718326, 0.76629378, 0.70980776, 0.1700194 ]))

In [0]:
projections = np.random.randn(2,5)

In [12]:
np.dot(vec1, projections.T)

array([-0.48402468, -0.38112615])